In [1]:
import os
from datetime import datetime


In [2]:
#set FIXED variables
HOMEPATH = "/home/ubuntu/fastai/"
DOGSCATSPATH = HOMEPATH + 'dogscats/'
MODELPATH = HOMEPATH + 'dogscats/models/'
TRAINPATH = HOMEPATH + 'dogscats/train/'
VALIDPATH = HOMEPATH + 'dogscats/valid/'
TESTPATH = HOMEPATH + 'dogscats/test1/'


In [31]:
from keras import backend as K
print (keras.__version__)
print (theano.__version__)
import tensorflow
print (tensorflow.__version__)

1.2.2
0.9.0.dev-c697eeab84e5b8a74908da654b66ec9eca4f1291
1.3.0


In [3]:

import os

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

Using TensorFlow backend.
Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


In [4]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [5]:
#owngrade version of keras
#pip install keras==1.2.2


os.chdir(HOMEPATH)
print ("current working directory:", os.getcwd())
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

('current working directory:', '/home/ubuntu/fastai')


## Setup

In [6]:
print ("MODELPATH:", MODELPATH)
if not os.path.exists(MODELPATH): os.mkdir(MODELPATH)

batch_size=64

MODELPATH: /home/ubuntu/fastai/dogscats/models/


In [7]:
batches = get_batches(TRAINPATH, shuffle=False, batch_size=batch_size)
val_batches = get_batches(VALIDPATH, shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [8]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(DOGSCATSPATH)
#nb: created duplicate directory of dogs/cats/test since test1 was used ealier.

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In this notebook we're going to create an ensemble of models and use their average as our predictions. For each ensemble, we're going to follow our usual fine-tuning steps:

1) Create a model that retrains just the last layer
2) Add this to a model containing all VGG layers except the last layer
3) Fine-tune just the dense layers of this model (pre-computing the convolutional layers)
4) Add data augmentation, fine-tuning the dense layers without pre-computation.

So first, we need to create our VGG model and pre-compute the output of the conv layers:

In [9]:
model = Vgg16().model
conv_layers,fc_layers = split_at(model, Convolution2D)

In [10]:
conv_model = Sequential(conv_layers)

In [11]:
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)
trn_features = conv_model.predict_generator(batches, batches.nb_sample)

In [12]:
save_array(MODELPATH + 'train_convlayer_features.bc', trn_features)
save_array(MODELPATH + 'valid_convlayer_features.bc', val_features)

In the future we can just load these precomputed features:

In [13]:
trn_features = load_array(MODELPATH+'train_convlayer_features.bc')
val_features = load_array(MODELPATH+'valid_convlayer_features.bc')

We can also save some time by pre-computing the training and validation arrays with the image decoding and resizing already done:

In [14]:
trn = get_data(TRAINPATH)
val = get_data(VALIDPATH)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [15]:
save_array(MODELPATH + 'train_data.bc', trn)
save_array(MODELPATH + 'valid_data.bc', val)

In the future we can just load these resized images:

In [16]:
startTime= datetime.now()
trn = load_array(MODELPATH + 'train_data.bc')
val = load_array(MODELPATH + 'valid_data.bc')
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


Time elpased (hh:mm:ss.ms) 0:00:06.944434


Finally, we can precompute the output of all but the last dropout and dense layers, for creating the first stage of the model:

In [17]:
startTime= datetime.now()
model.pop()
model.pop()
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


Time elpased (hh:mm:ss.ms) 0:00:00.000162


In [37]:
print (type(model))
print (type(model.layers))

for layer in model.layers:
    print (type(layer))

<class 'keras.models.Sequential'>
<type 'list'>
<class 'keras.layers.core.Lambda'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Convolution2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Convolution2D'>
<class 'keras.layers.pooling.MaxPooling2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Convolution2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Convolution2D'>
<class 'keras.layers.pooling.MaxPooling2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Convolution2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Convolution2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers.convolutional.Convolution2D'>
<class 'keras.layers.pooling.MaxPooling2D'>
<class 'keras.layers.convolutional.ZeroPadding2D'>
<class 'keras.layers

In [ ]:
startTime= datetime.now()
ll_val_feat = model.predict_generator(val_batches, val_batches.nb_sample)
ll_feat = model.predict_generator(batches, batches.nb_sample)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


In [19]:
startTime= datetime.now()
save_array(MODELPATH + 'train_ll_feat.bc', ll_feat)
save_array(MODELPATH + 'valid_ll_feat.bc', ll_val_feat)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


Time elpased (hh:mm:ss.ms) 0:00:00.729212


In [20]:
startTime= datetime.now()
ll_feat     = load_array(MODELPATH + 'train_ll_feat.bc')
ll_val_feat = load_array(MODELPATH + 'valid_ll_feat.bc')
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))


Time elpased (hh:mm:ss.ms) 0:00:00.272077


In [35]:
print (type(ll_feat), "ll_feat:", ll_feat.shape)
print (type(ll_val_feat), "ll_val_feat:", ll_val_feat.shape)

<type 'numpy.ndarray'> ll_feat: (23000, 4096)
<type 'numpy.ndarray'> ll_val_feat: (2000, 4096)


...and let's also grab the test data, for when we need to submit:

In [21]:
#this is wrong, since test data is not classified.
print ("TESTPATH:", TESTPATH)
test = get_data(TESTPATH)
print (type(test))
save_array(MODELPATH+'test_data.bc', test)

TESTPATH: /home/ubuntu/fastai/dogscats/test1/
Found 0 images belonging to 0 classes.


ValueError: need at least one array to concatenate

In [ ]:
test = load_array(MODELPATH+'test_data.bc')

## Last layer

The functions automate creating a model that trains the last layer from scratch, and then adds those new layers on to the main model.

In [22]:
def get_ll_layers():
    return [ 
        BatchNormalization(input_shape=(4096,)),
        Dropout(0.5),
        Dense(2, activation='softmax') 
        ]

In [23]:
def train_last_layer(i):
    ll_layers = get_ll_layers()
    ll_model = Sequential(ll_layers)
    ll_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    ll_model.optimizer.lr=1e-5
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), nb_epoch=12)
    ll_model.optimizer.lr=1e-7
    ll_model.fit(ll_feat, trn_labels, validation_data=(ll_val_feat, val_labels), nb_epoch=1)
    ll_model.save_weights(MODELPATH+'ll_bn' + i + '.h5')

    vgg = Vgg16()
    model = vgg.model
    model.pop(); model.pop(); model.pop()
    for layer in model.layers: layer.trainable=False
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    ll_layers = get_ll_layers()
    for layer in ll_layers: model.add(layer)
    for l1,l2 in zip(ll_model.layers, model.layers[-3:]):
        l2.set_weights(l1.get_weights())
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    model.save_weights(MODELPATH+'bn' + i + '.h5')
    return model

## Dense model

In [24]:
def get_conv_model(model):
    layers = model.layers
    last_conv_idx = [index for index,layer in enumerate(layers) 
                         if type(layer) is Convolution2D][-1]

    conv_layers = layers[:last_conv_idx+1]
    conv_model = Sequential(conv_layers)
    fc_layers = layers[last_conv_idx+1:]
    return conv_model, fc_layers, last_conv_idx

In [25]:
def get_fc_layers(p, in_shape):
    return [
        MaxPooling2D(input_shape=in_shape),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
        ]

In [26]:
def train_dense_layers(i, model):
    conv_model, fc_layers, last_conv_idx = get_conv_model(model)
    conv_shape = conv_model.output_shape[1:]
    fc_model = Sequential(get_fc_layers(0.5, conv_shape))
    for l1,l2 in zip(fc_model.layers, fc_layers): 
        weights = l2.get_weights()
        l1.set_weights(weights)
    fc_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                     metrics=['accuracy'])
    fc_model.fit(trn_features, trn_labels, nb_epoch=2, 
         batch_size=batch_size, validation_data=(val_features, val_labels))

    gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05, 
       width_zoom_range=0.05, zoom_range=0.05,
       channel_shift_range=10, height_shift_range=0.05, shear_range=0.05, horizontal_flip=True)
    batches = gen.flow(trn, trn_labels, batch_size=batch_size)
    val_batches = image.ImageDataGenerator().flow(val, val_labels, 
                      shuffle=False, batch_size=batch_size)

    for layer in conv_model.layers: layer.trainable = False
    for layer in get_fc_layers(0.5, conv_shape): conv_model.add(layer)
    for l1,l2 in zip(conv_model.layers[last_conv_idx+1:], fc_model.layers): 
        l1.set_weights(l2.get_weights())

    conv_model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', 
                       metrics=['accuracy'])
    conv_model.save_weights(MODELPATH+'no_dropout_bn' + i + '.h5')
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=1, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    for layer in conv_model.layers[16:]: layer.trainable = True
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=8, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)

    conv_model.optimizer.lr = 1e-7
    conv_model.fit_generator(batches, samples_per_epoch=batches.N, nb_epoch=10, 
                            validation_data=val_batches, nb_val_samples=val_batches.N)
    conv_model.save_weights(MODELPATH + 'aug' + i + '.h5')

## Build ensemble

In [27]:
startTime= datetime.now()
for i in range(5):
    i = str(i)
    model = train_last_layer(i)
    train_dense_layers(i, model)
timeElapsed=datetime.now()-startTime
print('Time elpased (hh:mm:ss.ms) {}'.format(timeElapsed))

Train on 23000 samples, validate on 2000 samples
Epoch 1/12
23000/23000 [==============================] - 3s - loss: 0.6873 - acc: 0.7441 - val_loss: 2.3140 - val_acc: 0.3805
Epoch 2/12
23000/23000 [==============================] - 3s - loss: 0.4753 - acc: 0.8509 - val_loss: 2.7220 - val_acc: 0.3675
Epoch 3/12
23000/23000 [==============================] - 3s - loss: 0.4501 - acc: 0.8701 - val_loss: 2.8306 - val_acc: 0.3670
Epoch 4/12
23000/23000 [==============================] - 3s - loss: 0.4445 - acc: 0.8752 - val_loss: 2.8214 - val_acc: 0.3685
Epoch 5/12
23000/23000 [==============================] - 3s - loss: 0.4319 - acc: 0.8770 - val_loss: 2.8307 - val_acc: 0.3680
Epoch 6/12
23000/23000 [==============================] - 3s - loss: 0.4204 - acc: 0.8822 - val_loss: 2.8126 - val_acc: 0.3685
Epoch 7/12
23000/23000 [==============================] - 3s - loss: 0.4077 - acc: 0.8847 - val_loss: 2.7669 - val_acc: 0.3680
Epoch 8/12
23000/23000 [==============================] - 3s -

ValueError: You called `set_weights(weights)` on layer "batchnormalization_3" with a  weight list of length 0, but the layer was expecting 4 weights. Provided weights: []...

## Combine ensemble and test

In [ ]:

ens_model = vgg_ft(2)
for layer in ens_model.layers: 
    layer.trainable=True


In [ ]:
def get_ens_pred(arr, fname):
    ens_pred = []
    for i in range(5):
        i = str(i)
        ens_model.load_weights('{}{}{}.h5'.format(MODELPATH, fname, i))
        preds = ens_model.predict(arr, batch_size=batch_size)
        ens_pred.append(preds)
    return ens_pred

In [ ]:
val_pred2 = get_ens_pred(val, 'aug')

In [ ]:
val_avg_preds2 = np.stack(val_pred2).mean(axis=0)

In [ ]:
categorical_accuracy(val_labels, val_avg_preds2).eval()